## 커스텀 레이어
##### url: https://www.tensorflow.org/tutorials/customization/custom_layers

- 목차
    - 레이어: 유용한 연산들의 일반적 집합
    - 커스텀 레이어 구현하기
    - 모델: 레이어로 구성되는 모델
     
신경망을 만들기 위해 고수준 API로써 `tf.keras`를 사용하기를 추천합니다.<br>
이는, 대부분의 Tensorflow API는 '즉시 실행'(eager execution)으로 사용할 수 있기 때문입니다.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

In [3]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


## 레이어: 유용한 연산들의 일반적인 집합
머신러닝 모델을 작성할 때 대부분의 경우, 독립적인 연산과 독립적인 변수를 다루는 것보다 더 높은 수준의 추상화로 작동하길 원합니다.<br><br>
많은 머신러닝 모델은 비교적 간단한 레이어의 적층이나 구성으로서 표현되어집니다. 또한, Tensorflow는 일반적으로 알려진 많은 레이어 뿐만아니라 처음부터 구현하거나 기존의 레이어의 조합으로써 사용자의 어플리케이션에 특정화된 레이어를 제공합니다. <br><br>
Tensorflow는 tf.keras 패키지에 Keras API의 전체를 포함하고 있으며, 모델을 작성할 때 Keras 레이어는 매우 유용합니다.

In [4]:
# tf.keras.layers 패키지에 들어있는 레이어는 객체입니다. 레이어를 생성하기 위해서는,
# 간단히 객체를 생성하면 됩니다. 대부분의 레이어는 첫 번째 인자로써 출력 차원의 수 / 채널을 입력받습니다.
layer = tf.keras.layers.Dense(100)
# 입력 차원의 수는 종종 필요하지 않을 수 있습니다. 레이어가 처음 사용되어질 때
# 추론될 수 있기 때문입니다. 하지만 수동으로 입력해서 해당 값을 제공할 수 있다면
# 모델의 복잡도에 도움이될 수 있습니다.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

존재하는 레이어의 전체 목록은 [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers) 에서 확인할 수 있습니다. Dense(a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout 등과 같은 많은 레이어가 있습니다.

In [5]:
# 레이어를 사용하기 위해서는 간단하게 호출하면 됩니다.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [6]:
# 레이어는 많은 유용한 메소드를 가지고 있습니다. 예를 들어, `layer.variable`과 `layer.trainable_variables`를 사용하는 학습 가능한 변수를
# 확인할 수 있습니다, fully-connected layer의 경우 가중치와 편향 값을 가지고 있습니다.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.34981078,  0.03152984, -0.16619813, -0.06974012,  0.58996636,
          0.5936511 ,  0.5167751 ,  0.0053215 ,  0.45745295,  0.5192371 ],
        [-0.5580803 ,  0.49557835, -0.33167747, -0.3998595 , -0.46926486,
         -0.6169045 , -0.25557932,  0.4935425 ,  0.08498222,  0.4698686 ],
        [ 0.521118  , -0.07488418,  0.17968726, -0.05080914, -0.04037559,
          0.3732043 ,  0.62010545, -0.05951196,  0.10654658,  0.14854783],
        [-0.088745  , -0.09918517, -0.2578876 ,  0.3918671 ,  0.31084627,
          0.58950263,  0.12035823, -0.3058497 ,  0.26442564,  0.19211745],
        [ 0.13619655, -0.2607354 , -0.28657085, -0.38408667,  0.10284948,
         -0.08702493,  0.09807962, -0.26731992, -0.11878413, -0.15013668]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [7]:
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.34981078,  0.03152984, -0.16619813, -0.06974012,  0.58996636,
          0.5936511 ,  0.5167751 ,  0.0053215 ,  0.45745295,  0.5192371 ],
        [-0.5580803 ,  0.49557835, -0.33167747, -0.3998595 , -0.46926486,
         -0.6169045 , -0.25557932,  0.4935425 ,  0.08498222,  0.4698686 ],
        [ 0.521118  , -0.07488418,  0.17968726, -0.05080914, -0.04037559,
          0.3732043 ,  0.62010545, -0.05951196,  0.10654658,  0.14854783],
        [-0.088745  , -0.09918517, -0.2578876 ,  0.3918671 ,  0.31084627,
          0.58950263,  0.12035823, -0.3058497 ,  0.26442564,  0.19211745],
        [ 0.13619655, -0.2607354 , -0.28657085, -0.38408667,  0.10284948,
         -0.08702493,  0.09807962, -0.26731992, -0.11878413, -0.15013668]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## 커스텀 레이어 구현하기
자신만의 레이어를 구현하는 가장 좋은 방법은 tf.keras.Layer 클래스를 확장하고 구현하는 것 입니다.<br>
1. `__init__`, 입력에 독립적으로 초기화할 수 있습니다.
2. `build`, 입력 텐서의 차원을 알게되거나, 초기화를 진행할 수 있습니다.
3. `call`, 순방향으로 연산을 진행합니다. 
변수를 생성하기 위해 `build`가 호출될 때까지 기다릴 수 없다는 것을 명심해야합니다. `__init__` 함수 내에서 생성할 수 있습니다.<br>
하지만, 변수를 `build`내에서 생성하는 것에 대한 이점은 레이어가 작동할 때 들어오는 입력의 차원을 기반으로 변수를 생성할 수 있다는 점 입니다.<br>
다르게 말하면, `__init__`에서 변수를 생성할 경우에는 미리 생성할 변수의 차원을 알고 있어야 한다는 의미입니다.

In [8]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_weight('kernel',
                                      shape=[int(input_shape[-1]),
                                             self.num_outputs])
    
    def call(self, input):
        return tf.matmul(input, self.kernel)
    
layer = MyDenseLayer(10)

In [9]:
_ = layer(tf.zeros([10, 5])) # 레이어를 호출 '.builds' 

In [10]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


가능한 표준 레이어를 사용한다면, 전체 코드는 가독성과 유지보수하기 용이해집니다. 코드를 읽는 다른 사람들은 표준 레이어의 동작 방식에 익숙하기 때문입니다.<br>
만약 `tf.keras.layers`에 존재하지 않는 레이어를 사용한다면, github 이슈를 남기는 것을 고려해보는것이 좋습니다. 풀리퀘스트를 보내주면 더 좋겠네요.